In [1]:
import sys 
import pandas as pd
import logging

sys.path.append('../')

import utils.llm 
import utils.scoring

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

class Config:
    DATA_DIR = '../data/'
    DATA_OUT_DIR = '../data_out/'
    TEMPERATURE = 0.01
    SAMPLE_N = 30
    
    # 収集対象の対象クイズの範囲
    TARGET_QUIZ_RANGE = [i for i in range(1, 201)]
    


annotated_df = pd.read_csv(Config.DATA_DIR + 'df_JAQKET.csv')





In [2]:
llm = utils.llm.LLM()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Loading binary /home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ryoryon66/.pyenv/versions/3.9.8/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


In [3]:
def sampling(query:str,temperature:float,sample_n:int) -> list[str]:
    
    sampled = []
    
    for i in range(sample_n):
        
        q = query
        sampled.append(llm.generate(q,temperature))
    
    logger.info("sampled: {}".format(sampled))
    
    return sampled


In [4]:
# serial_number, qid, query, responseを記録するdf

df = pd.DataFrame(columns=['serial_number','qid','query','sample'])


serial_number = 0



for i in Config.TARGET_QUIZ_RANGE:
    
    # show progress
    if i % 10 == 0:
        logger.info("now processing the quiz of index... {}".format(i))
    
    qid = annotated_df.iloc[i]['qid']
    query = annotated_df.iloc[i]['question']
    response = annotated_df.iloc[i]['generated_answer']
    
    sampled = sampling(query,Config.TEMPERATURE,Config.SAMPLE_N)
    
    for sample in sampled:
        
        #  The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
        
        df = pd.concat([df,pd.DataFrame([[serial_number,qid,query,sample]],columns=['serial_number','qid','query','sample'])])
        
        serial_number += 1


sampled: ['『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。', '『東方見聞録』です。']
sampled: ['村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹', '村上春樹']
sampled: ['ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal Kick)といいます。', 'ゴールキック(Goal 

In [5]:
df.shape

(6000, 4)

In [6]:
import datetime

info = "temperature_{}_sample_n_{}".format(Config.TEMPERATURE,Config.SAMPLE_N)
suffix = datetime.datetime.now().strftime('%m_%d_%H_%M_%S')

df.to_csv(Config.DATA_OUT_DIR + 'same_query_{}_{}.csv'.format(info,suffix),index=False)




In [7]:
df

,serial_number,qid,query,sample
0,0,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,『東方見聞録』です。
0,1,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,『東方見聞録』です。
0,2,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,『東方見聞録』です。
0,3,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,『東方見聞録』です。
0,4,QA20CAPR-0003,作家のルスティケロが、マルコ・ポーロから聞いた話をまとめた作品といえば何でしょう?,『東方見聞録』です。
...,...,...,...,...
0,5995,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦芽
0,5996,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦芽
0,5997,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦芽
0,5998,QA20CAPR-0405,味噌や醤油の原料になる六条と、ビールやウイスキーの原料になる二条に大別されるイネ科の植物は何...,米麦芽
